In [ ]:
import pandas as pd
import numpy as np
import re

 
pre_path = '/Users/jes/Library/CloudStorage/OneDrive-RobertGordonUniversity(2)/ProjectMINE/Birmingham/AirF_Pre.csv'
post_path = '/Users/jes/Library/CloudStorage/OneDrive-RobertGordonUniversity(2)/ProjectMINE/Birmingham/AirF_Post.csv'
 
#  Function to clean each air dataset
 
def clean_air_file(file_path, period_label):
    """Load, clean, and reshape air quality data."""
    df = pd.read_csv(file_path, encoding='latin1')
    
    # --- Combine date and time ---
    df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], errors='coerce')
    df = df.drop(columns=['Date', 'Time'])
    
    # --- Select only Nitrogen dioxide columns (ignore Status) ---
    no2_cols = [c for c in df.columns if 'Nitrogen dioxide' in c]
    df = df[['Datetime'] + no2_cols]
    
    # --- Melt to long format ---
    long_df = df.melt(id_vars='Datetime', var_name='site_name', value_name='raw_no2')
    
    # --- Clean site names (remove "Nitrogen dioxide")  
    long_df['site_name'] = (
        long_df['site_name']
        .str.replace('Nitrogen dioxide', '', regex=False)
        .str.replace('-', '')
        .str.strip()
    )
    
    # --- Convert raw values to numbers (ignore "No data") ---
    def to_float(x):
        if pd.isna(x): return np.nan
        s = str(x).lower()
        if 'no data' in s: return np.nan
        m = re.search(r'[-+]?\d*\.\d+|\d+', s)
        return float(m.group(0)) if m else np.nan
    
    long_df['NO2'] = long_df['raw_no2'].apply(to_float)
    
    # --- Add period label (Pre/Post) ---
    long_df['Period'] = period_label
    
    # --- Clean final structure ---
    long_df = long_df[['Datetime', 'site_name', 'NO2', 'Period']]
    long_df = long_df.dropna(subset=['Datetime', 'NO2'])
    long_df = long_df.drop_duplicates(subset=['Datetime', 'site_name'])
    
    return long_df

 
#  Apply function to both datasets
 
air_pre_clean  = clean_air_file(pre_path,  'Pre-CAZ')
air_post_clean = clean_air_file(post_path, 'Post-CAZ')

 
#  Save outputs separately
 
air_pre_clean.to_csv('/Users/jes/Library/CloudStorage/OneDrive-RobertGordonUniversity(2)/ProjectMINE/Birmingham/Air_Pre_Clean.csv', index=False)
air_post_clean.to_csv('/Users/jes/Library/CloudStorage/OneDrive-RobertGordonUniversity(2)/ProjectMINE/Birmingham/Air_Post_Clean.csv', index=False)

print('✅ Air data cleaned and saved.')
print('Pre shape:', air_pre_clean.shape, 'Post shape:', air_post_clean.shape)
print('\nSample:')
print(air_pre_clean.head())


✅ Air data cleaned and saved.
Pre shape: (32037, 4) Post shape: (34580, 4)

Sample:
             Datetime site_name      NO2   Period
0 2018-01-03 01:00:00     A4540  9.13899  Pre-CAZ
1 2018-01-03 02:00:00     A4540  6.98991  Pre-CAZ
2 2018-01-03 03:00:00     A4540  7.15747  Pre-CAZ
3 2018-01-03 04:00:00     A4540  6.70567  Pre-CAZ
4 2018-01-03 05:00:00     A4540  8.26296  Pre-CAZ
